# Le tour de France des musées

In [1]:
# libraries
import pandas as pd
import numpy as np

In [2]:
# load data
df = pd.read_csv('data.csv', delimiter=';', encoding='latin1')

___

### Appercu des données (5 entrées aléatoires)

In [3]:
df.sample(5, random_state=42)

,osm_id,name,number,street,postal_code,city,country,country_code,lat,lon,website,email,phone,fax,tags,description,date_added,wikidata
1578,4333063561,Musé du Vitrail,NaN,Route de Sanxay,86600.0,Curzay-sur-Vonne,France,fr,46.490902,0.043856,http://www.musee-du-vitrail.com/,NaN,NaN,NaN,osm:museum;type:musee technique et industriel,NaN,"42585,0242939815",NaN
3061,5827811617,Musée Gertrude Schoen,NaN,Impasse du Donjon,47340.0,Laroque-Timbaut,France,fr,44.282700,0.760782,NaN,NaN,NaN,NaN,osm:museum,NaN,"43324,9078240741",NaN
1557,930071765,Moulin de La Rouzique,NaN,Chemin des Guillandoux,24150.0,Couze-et-Saint-Front,France,fr,44.829980,0.702082,NaN,NaN,NaN,NaN,osm:museum;type:ecomusee,NaN,"41408,7629513889",NaN
465,95358738,Ecomusée de l'Avesnois,NaN,Place Maria Blondeau,59612.0,Fourmies,France,fr,50.017585,4.042354,http://www.ecomusee-avesnois.fr/,NaN,NaN,NaN,osm:museum;type:ecomusee,NaN,"43007,1936921296",Q3578600
776,65742737,Hôtel des Sires de Neuchâtel,NaN,Place du Général de Gaulle,25110.0,Baume-les-Dames,France,fr,47.352715,6.360958,NaN,NaN,NaN,NaN,osm:museum,NaN,"43043,4666435185",Q3146262


### Combien y-a-t-il de musées en France métropolitaine ?

In [30]:
df.shape[0]

3784

___

### Dans quelle(s) ville(s) y-a-t-il de plus de musées ?

In [56]:
serie = df.groupby(['city'])['city'].count().sort_values(ascending=False)

In [66]:
serie[serie==serie[0]].index[0]

'Paris'

In [67]:
serie[0]

175

#### Paris est la ville comprenant le plus de musées (175)

___

### Quel est le nombre moyen de musées par ville ?

In [71]:
# Le 1er janvier 2022, la France compte : 
# 34 826 communes en métropole dans les 96 départements métropolitains et le Grand Lyon
# (Wikipedia : https://fr.wikipedia.org/wiki/Nombre_de_communes_en_France)

Ici on divise le nombre de musées en France Métropolitaine par son nombre de villes.
le calcul np.mean(serie) aurait mené à une erreur de biais de séléction, car il n'existe peut-être pas d'entrée pour des villes n'ayant pas de musée

In [74]:
np.sum(serie)/34826

0.10859702521104922

In [76]:
# mauvais résultat :
np.mean(serie)

1.647930283224401

___

### Quel est le nombre médian de musées par ville ?

Avec la même logique que précédemment, il y a 3784 villes avec un musée en France Métropolitaine, pour 34826 communes. la médiane est évidemment 0

In [78]:
0

0

In [77]:
# mauvais résultat :
np.median(serie)

1.0

___

### Comment sont répartis les musées par type (en pourcentage) ?

In [80]:
# le type de musée se trouve dans la colonne tag, qui semble être de la forme tag:text;tag:text
# il faut parser cette colonne

In [108]:
def extract_tag(tags):
    types = [t.split(':')[-1] for t in str(tags).split(';') if 'type' in t]
    if len(types) == 0:
        return None
    return types[0]

vextract = np.vectorize(extract_tag)

In [121]:
df['type'] = pd.Series(vextract(df['tags']))

In [122]:
df['type'].dropna().value_counts()

ecomusee                         387
musee technique et industriel    209
musee d'histoire                 156
musee de beaux-arts              117
musee archeologique              107
musee d'art                       84
maison d'artiste                  66
musee d'histoire naturelle        57
musee d'arts populaires           52
centre d'interpretation           46
musee de sciences                 22
centre d'art                      20
fondation                         18
musee d'artiste                   15
musee litteraire                   9
atelier d'artiste                  8
bibliotheque                       4
musee ethnographique               3
musée de beaux-arts                2
musee d'architecture               1
ecomuseum                          1
Name: type, dtype: int64

#### Ici nous avons les valeurs absolues. maintenant, en pourcentage : 

In [127]:
df['type'].dropna().value_counts(normalize=True) * 100

ecomusee                         27.962428
musee technique et industriel    15.101156
musee d'histoire                 11.271676
musee de beaux-arts               8.453757
musee archeologique               7.731214
musee d'art                       6.069364
maison d'artiste                  4.768786
musee d'histoire naturelle        4.118497
musee d'arts populaires           3.757225
centre d'interpretation           3.323699
musee de sciences                 1.589595
centre d'art                      1.445087
fondation                         1.300578
musee d'artiste                   1.083815
musee litteraire                  0.650289
atelier d'artiste                 0.578035
bibliotheque                      0.289017
musee ethnographique              0.216763
musée de beaux-arts               0.144509
musee d'architecture              0.072254
ecomuseum                         0.072254
Name: type, dtype: float64

___

### Combien y-a-t-il de musées dont le nom commence par "Château" ?

In [131]:
df[df['name'].str.startswith('Château')].shape[0]

57

___

### Pour combien de musées dispose-t-on de l'adresse du site web ?

In [135]:
df['email'].dropna().count()

301

___

### Quel département français possède le plus de musées sur son territoire ?

In [151]:
def dept(postal_code):
    return postal_code[0:2]
vdept = np.vectorize(dept)

'65'

In [152]:
df['dept'] = vdept(df['postal_code'])

In [159]:
df['dept'].value_counts()

75    180
59     97
33     94
71     91
13     90
     ... 
28     14
23     11
90     10
48      9
98      8
Name: dept, Length: 88, dtype: int64

Ici nous découvrons que le département 98 qui n'est pas en France métropolitaine s'est glissé dans nos data.
cela ne fausse pas sensiblement les résultats obtenus précédemment

#### Département avec le plus de musées :

Paris

#### Département avec le moins de musées:

Lozère

___

### Combien de musées ont "Napoléon" dans leur nom ?

In [167]:
df[df['name'].str.find('Napoléon') != -1]

,osm_id,name,number,street,postal_code,city,country,country_code,lat,lon,...,email,phone,fax,tags,description,date_added,wikidata,type,departement,dept
3259,3678466279,Musée Napoléon,NaN,D 32,nan,Périgueux,France,fr,44.974544,0.837015,...,NaN,NaN,NaN,osm:museum,NaN,"43811,9434027778",Q2178225,None,na,na
3260,64991060,Musée Napoléon,NaN,Rue des Remparts,17123,Île-d'Aix,France,fr,46.013011,-1.174656,...,NaN,3.354685e+10,NaN,osm:museum,NaN,"43546,4584375",NaN,None,17,17
3261,480527098,Musée Napoléon Ier,NaN,Rue de la Perrière,21220,Fixin,France,fr,47.245420,4.962403,...,NaN,3.338052e+10,NaN,osm:museum,NaN,"43927,607962963",NaN,None,21,21
3262,7820341496,Musée Napoléon Ier et Trésors des Églises,NaN,Rue du Maréchal Valée,10500,Brienne-le-Château,France,fr,48.389077,4.527967,...,NaN,3.332593e+10,NaN,osm:museum,NaN,"44197,0378356481",NaN,None,10,10


In [168]:
4

4

_____

_____

## Proposer deux axes d'analyse supplémentaires

Nous pouvons utiliser l'estimation des temps de parcours en TGV (https://ressources.data.sncf.com/explore/dataset/meilleurs-temps-des-parcours-des-trains/table/?sort=annee)
pour proposer des circuits reliant des musées avec des thèmes communs